# RoBERTa ile Veri Setinden Öznitelik Çıkarımı

Öncelikle MELD veri setini yükleyip tokenize ederek işleme hazır hale getirdim.

In [32]:
import pandas as pd

# MELD veri setini yükle
meld_dev = pd.read_csv("../../data/raw/dev_sent_emo.csv")

# İlgili sütunları al
meld_texts = meld_dev["Utterance"].tolist() # kullandım
meld_labels = meld_dev["Emotion"] # kullandım
meld_speaker = meld_dev["Speaker"].tolist()
meld_conversation = meld_dev["Dialogue_ID"].tolist()

In [33]:
print(meld_dev.head(1))

   Sr No.                                       Utterance Speaker  Emotion  \
0       1  Oh my God, hes lost it. Hes totally lost it.  Phoebe  sadness   

  Sentiment  Dialogue_ID  Utterance_ID  Season  Episode     StartTime  \
0  negative            0             0       4        7  00:20:57,256   

        EndTime  
0  00:21:00,049  


RoBERTa tokenizer ile metinleri işledim.

In [34]:
from transformers import RobertaTokenizer

# Tokenizer'ı yükle
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

# Bütün metinleri tokenize et
tokens = tokenizer(
    meld_texts, 
    padding="max_length",  # Maksimum uzunluğa göre pad ekler
    truncation=True,       # Uzun metinleri keser
    max_length=128,        # Maksimum uzunluk
    return_tensors="pt"    # PyTorch tensörü olarak döndürür
)

print(tokens["input_ids"].shape)  # (num_samples, 128)
print(tokens)


torch.Size([1109, 128])
{'input_ids': tensor([[    0,  7516,   127,  ...,     1,     1,     1],
        [    0,  2264,   116,  ...,     1,     1,     1],
        [    0, 11094,   328,  ...,     1,     1,     1],
        ...,
        [    0,   100,  4056,  ...,     1,     1,     1],
        [    0,   100,  1266,  ...,     1,     1,     1],
        [    0,   100,   216,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


Hugging Face kütüphanesini kullanarak RoBERTa gömme özniteliklerini çıkardım.

In [35]:
from transformers import RobertaModel
import torch

# RoBERTa modelini yükle
model = RobertaModel.from_pretrained("roberta-large")

# Model ile embedding çıkartma
with torch.no_grad():
    outputs = model(**tokens)

# Son gizli katman çıktısı (CLS token embedding'i kullan)
roberta_features = outputs.last_hidden_state[:, 0, :]  # (batch_size, hidden_dim)

# Özellik boyutu
print(roberta_features.shape)  # (num_samples, 768)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1109, 1024])


DialogueRNN modeli, konuşmaları sıralı olarak işlediği için veriyi konuşmalara göre gruplayıp işlemek gerekiyor.
Her konuşmanın diyalog sırasını koruyarak bir liste formatına çevirdim.

In [36]:
from collections import defaultdict

# Diyalogları gruplama
dialogues = defaultdict(list)
for text, speaker, conv_id, feature, label in zip(meld_texts, meld_speaker, meld_conversation, roberta_features, meld_labels):
    dialogues[conv_id].append((feature, speaker, label))

# Sıralı diyalog listesi oluştur
sorted_dialogues = [dialogues[key] for key in sorted(dialogues.keys())]


Veri seti içerisindeki duygu etiketleri string veri tipinde olduğundan bunları one hot encoding yöntemi ile temsil ettim.

In [37]:
import numpy as np

# Duygu etiketleri için one-hot encoding dönüşümü
def one_hot_encode(label, num_classes = 7):
    one_hot = np.zeros(num_classes)
    one_hot[label] = 1
    return one_hot

# Sözlük yapısı kullanarak duygu eitketlerinin rakam temsilini oluşturma
emotion_dict = {
    "neutral": 0,
    "joy": 1,
    "sadness": 2,
    "anger": 3,
    "fear": 4,
    "disgust": 5,
    "surprise": 6
}

# Etiket sütununu önce sayısal daha sonra one-hot encode formatına getirme
meld_labels = meld_labels.map(emotion_dict)
meld_labels = meld_labels.apply(lambda x: one_hot_encode(x))
print(type(meld_labels))

<class 'pandas.core.series.Series'>


In [38]:
# meld_labels=meld_labels.tolist
# print(type(meld_labels))

## DialogueRNN Modelini Eğitme ve Test Etme
DialogueRNN modelini kurup, RoBERTa’dan gelen embedding’leri giriş olarak verdim. 

In [41]:
import torch.nn as nn

class DialogueRNN(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=128, num_classes=7):
        super(DialogueRNN, self).__init__()
        
        self.rnn = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        output = self.fc(rnn_out)
        return output

# Modeli oluştur
dialoguernn = DialogueRNN()

Şimdi modeli eğitmek için verileri batch halinde besleyip çapraz entropi kaybı ile eğittim.

### Aşağıdaki kod hatalı ama not olarak kalsın

In [45]:
import torch.optim as optim

# Cihazı belirle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dialoguernn.to(device)

# Kayıp fonksiyonu ve optimizasyon

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(dialoguernn.parameters(), lr=0.001)

# Eğitim döngüsü
num_epochs = 5
for epoch in range(num_epochs):
    total_loss = 0
    
    for conversation in sorted_dialogues:
        features, speakers, labels = zip(*conversation)
        features = torch.stack(features).to(device)
        labels = torch.tensor(np.stack(labels), dtype=torch.float).to(device)
        
        # Modeli çalıştır
        outputs = dialoguernn(features)
        loss = criterion(outputs, labels)
        
        # Geri yayılım
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [28]:
for i, label in enumerate(meld_labels):
    print(f"Index {i}: {label} (Type: {type(label)})")


Index 0: [0. 0. 1. 0. 0. 0. 0.] (Type: <class 'numpy.ndarray'>)
Index 1: [0. 0. 0. 0. 0. 0. 1.] (Type: <class 'numpy.ndarray'>)
Index 2: [1. 0. 0. 0. 0. 0. 0.] (Type: <class 'numpy.ndarray'>)
Index 3: [0. 1. 0. 0. 0. 0. 0.] (Type: <class 'numpy.ndarray'>)
Index 4: [0. 0. 1. 0. 0. 0. 0.] (Type: <class 'numpy.ndarray'>)
Index 5: [1. 0. 0. 0. 0. 0. 0.] (Type: <class 'numpy.ndarray'>)
Index 6: [1. 0. 0. 0. 0. 0. 0.] (Type: <class 'numpy.ndarray'>)
Index 7: [0. 1. 0. 0. 0. 0. 0.] (Type: <class 'numpy.ndarray'>)
Index 8: [1. 0. 0. 0. 0. 0. 0.] (Type: <class 'numpy.ndarray'>)
Index 9: [0. 0. 0. 0. 0. 0. 1.] (Type: <class 'numpy.ndarray'>)
Index 10: [1. 0. 0. 0. 0. 0. 0.] (Type: <class 'numpy.ndarray'>)
Index 11: [1. 0. 0. 0. 0. 0. 0.] (Type: <class 'numpy.ndarray'>)
Index 12: [0. 0. 0. 0. 0. 0. 1.] (Type: <class 'numpy.ndarray'>)
Index 13: [0. 0. 0. 1. 0. 0. 0.] (Type: <class 'numpy.ndarray'>)
Index 14: [1. 0. 0. 0. 0. 0. 0.] (Type: <class 'numpy.ndarray'>)
Index 15: [0. 1. 0. 0. 0. 0. 0.] (T

In [46]:
import torch
import numpy as np
import pandas as pd
from transformers import RobertaTokenizer, RobertaModel
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict

# MELD veri setini yükle
meld_dev = pd.read_csv("../../data/raw/dev_sent_emo.csv")

# İlgili sütunları al
meld_texts = meld_dev["Utterance"].tolist()  # Kullandım
meld_labels = meld_dev["Emotion"]  # Kullandım
meld_speaker = meld_dev["Speaker"].tolist()
meld_conversation = meld_dev["Dialogue_ID"].tolist()

# Tokenizer'ı yükle
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

# Bütün metinleri tokenize et
tokens = tokenizer(
    meld_texts, 
    padding="max_length",  # Maksimum uzunluğa göre pad ekler
    truncation=True,       # Uzun metinleri keser
    max_length=128,        # Maksimum uzunluk
    return_tensors="pt"    # PyTorch tensörü olarak döndürür
)

# RoBERTa modelini yükle
model = RobertaModel.from_pretrained("roberta-large")

# Model ile embedding çıkartma
with torch.no_grad():
    outputs = model(**tokens)

# Son gizli katman çıktısı (CLS token embedding'i kullan)
roberta_features = outputs.last_hidden_state[:, 0, :]  # (batch_size, hidden_dim)

# Diyalogları gruplama
dialogues = defaultdict(list)
for text, speaker, conv_id, feature, label in zip(meld_texts, meld_speaker, meld_conversation, roberta_features, meld_labels):
    dialogues[conv_id].append((feature, speaker, label))

# Sıralı diyalog listesi oluştur
sorted_dialogues = [dialogues[key] for key in sorted(dialogues.keys())]

# Duygu etiketleri için one-hot encoding dönüşümü
def one_hot_encode(label, num_classes=7):
    one_hot = np.zeros(num_classes)
    one_hot[label] = 1
    return one_hot

# Sözlük yapısı kullanarak duygu etiketlerinin rakam temsilini oluşturma
emotion_dict = {
    "neutral": 0,
    "joy": 1,
    "sadness": 2,
    "anger": 3,
    "fear": 4,
    "disgust": 5,
    "surprise": 6
}

# Etiket sütununu önce sayısal daha sonra one-hot encode formatına getirme
meld_labels = meld_labels.map(emotion_dict)
meld_labels = meld_labels.apply(lambda x: one_hot_encode(x))  # One-hot encoding

# Model
class DialogueRNN(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=128, num_classes=7):
        super(DialogueRNN, self).__init__()
        
        self.rnn = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        output = self.fc(rnn_out)
        return output

# Modeli oluştur
dialoguernn = DialogueRNN()

# Cihazı belirle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dialoguernn.to(device)

# Kayıp fonksiyonu ve optimizasyon
criterion = nn.BCEWithLogitsLoss()  # One-hot encoding için uygun
optimizer = optim.Adam(dialoguernn.parameters(), lr=0.001)

# Eğitim döngüsü
num_epochs = 5
for epoch in range(num_epochs):
    total_loss = 0
    
    for conversation in sorted_dialogues:
        features, speakers, labels = zip(*conversation)
        features = torch.stack(features).to(device)
        
        # One-hot encoding'li etiketleri tensor'a dönüştür
        labels = torch.tensor(np.stack([one_hot_encode(emotion_dict[label]) for label in labels]), dtype=torch.float).to(device)
        
        # Modeli çalıştır
        outputs = dialoguernn(features)
        
        # Kayıp fonksiyonunu hesapla
        loss = criterion(outputs, labels)  # Artık boyut hatası vermez
        
        # Geri yayılım
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 41.3573
Epoch 2, Loss: 38.5827
Epoch 3, Loss: 38.0980
Epoch 4, Loss: 37.2894
Epoch 5, Loss: 36.3910
